In [13]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [14]:
CHUNKSIZE = 100000

TRAIN_PATH        = "../train_StationPathInfo.csv"
TRAIN_PATH_EX     = "../train_StationPathInfoEx.csv"
TEST_PATH         = "../test_StationPathInfo.csv"
TEST_PATH_EX      = "../test_StationPathInfoEx.csv"

TRAIN_DATE        = "../train_date.csv"
TRAIN_NUMERIC     = "../train_numeric.csv"
TRAIN_CATEGORICAL = "../train_categorical.csv"
TEST_DATE         = "../test_date.csv"
TEST_NUMERIC      = "../test_numeric.csv"
TEST_CATEGORICAL  = "../test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
    
    
    

In [15]:
chunk_train_num = pd.read_csv(TRAIN_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_num  = pd.read_csv(TEST_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_train_cat = pd.read_csv(TRAIN_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_cat  = pd.read_csv(TEST_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)

df_train_num = chunk_train_num.get_chunk(10)
df_test_num  = chunk_test_num.get_chunk(10)
df_train_cat = chunk_train_cat.get_chunk(10)
df_test_cat  = chunk_test_cat.get_chunk(10)

df_train_num.head()


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [16]:
df_train_cat.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
STATION_NUM = 52

num_cols = df_train_num.columns
cat_cols = df_train_cat.columns

num_feature_list = []
cat_feature_list = []

for cur_station_id in range(STATION_NUM):
    num_feature_list_one = ['Id']
    cat_feature_list_one = ['Id']
    for column_name in num_cols:
        if column_name != 'Id':
            r = parse("L{}_S{}_F{}", column_name)
            if r is not None:
                col_line_id    = int(r[0])
                col_station_id = int(r[1])
                if cur_station_id == col_station_id:
                    num_feature_list_one.append(column_name)
    for column_name in cat_cols:
        if column_name != 'Id':
            r = parse("L{}_S{}_F{}", column_name)
            if r is not None:
                col_line_id    = int(r[0])
                col_station_id = int(r[1])
                if cur_station_id == col_station_id:
                    cat_feature_list_one.append(column_name)
        
    num_feature_list.append(num_feature_list_one)
    cat_feature_list.append(cat_feature_list_one)


In [18]:
num_feature_list

[['Id',
  'L0_S0_F0',
  'L0_S0_F2',
  'L0_S0_F4',
  'L0_S0_F6',
  'L0_S0_F8',
  'L0_S0_F10',
  'L0_S0_F12',
  'L0_S0_F14',
  'L0_S0_F16',
  'L0_S0_F18',
  'L0_S0_F20',
  'L0_S0_F22'],
 ['Id', 'L0_S1_F24', 'L0_S1_F28'],
 ['Id',
  'L0_S2_F32',
  'L0_S2_F36',
  'L0_S2_F40',
  'L0_S2_F44',
  'L0_S2_F48',
  'L0_S2_F52',
  'L0_S2_F56',
  'L0_S2_F60',
  'L0_S2_F64'],
 ['Id',
  'L0_S3_F68',
  'L0_S3_F72',
  'L0_S3_F76',
  'L0_S3_F80',
  'L0_S3_F84',
  'L0_S3_F88',
  'L0_S3_F92',
  'L0_S3_F96',
  'L0_S3_F100'],
 ['Id', 'L0_S4_F104', 'L0_S4_F109'],
 ['Id', 'L0_S5_F114', 'L0_S5_F116'],
 ['Id', 'L0_S6_F118', 'L0_S6_F122', 'L0_S6_F132'],
 ['Id', 'L0_S7_F136', 'L0_S7_F138', 'L0_S7_F142'],
 ['Id', 'L0_S8_F144', 'L0_S8_F146', 'L0_S8_F149'],
 ['Id',
  'L0_S9_F155',
  'L0_S9_F160',
  'L0_S9_F165',
  'L0_S9_F170',
  'L0_S9_F175',
  'L0_S9_F180',
  'L0_S9_F185',
  'L0_S9_F190',
  'L0_S9_F195',
  'L0_S9_F200',
  'L0_S9_F205',
  'L0_S9_F210'],
 ['Id',
  'L0_S10_F219',
  'L0_S10_F224',
  'L0_S10_F229',
  'L0

S24-S30 はやたらと feature が多い。
S42, S46 は numeric feature がない。


In [19]:
cat_feature_list

[['Id'],
 ['Id', 'L0_S1_F25', 'L0_S1_F27', 'L0_S1_F29', 'L0_S1_F31'],
 ['Id',
  'L0_S2_F33',
  'L0_S2_F35',
  'L0_S2_F37',
  'L0_S2_F39',
  'L0_S2_F41',
  'L0_S2_F43',
  'L0_S2_F45',
  'L0_S2_F47',
  'L0_S2_F49',
  'L0_S2_F51',
  'L0_S2_F53',
  'L0_S2_F55',
  'L0_S2_F57',
  'L0_S2_F59',
  'L0_S2_F61',
  'L0_S2_F63',
  'L0_S2_F65',
  'L0_S2_F67'],
 ['Id',
  'L0_S3_F69',
  'L0_S3_F71',
  'L0_S3_F73',
  'L0_S3_F75',
  'L0_S3_F77',
  'L0_S3_F79',
  'L0_S3_F81',
  'L0_S3_F83',
  'L0_S3_F85',
  'L0_S3_F87',
  'L0_S3_F89',
  'L0_S3_F91',
  'L0_S3_F93',
  'L0_S3_F95',
  'L0_S3_F97',
  'L0_S3_F99',
  'L0_S3_F101',
  'L0_S3_F103'],
 ['Id',
  'L0_S4_F105',
  'L0_S4_F107',
  'L0_S4_F108',
  'L0_S4_F110',
  'L0_S4_F112',
  'L0_S4_F113'],
 ['Id'],
 ['Id',
  'L0_S6_F119',
  'L0_S6_F121',
  'L0_S6_F123',
  'L0_S6_F125',
  'L0_S6_F126',
  'L0_S6_F128',
  'L0_S6_F129',
  'L0_S6_F131',
  'L0_S6_F133',
  'L0_S6_F135'],
 ['Id'],
 ['Id'],
 ['Id',
  'L0_S9_F151',
  'L0_S9_F153',
  'L0_S9_F154',
  'L0_S9_F156

S24-S30はやたらとfeatureが多い。S0,S48他、いくつか categorical feature がない station がある。

In [28]:
# feature を抽出してファイルを作る。
for cat_feature_one in cat_feature_list:
    if 1 < len(cat_feature_one):
        r = parse("L{}_S{}_F{}", cat_feature_one[1])
        station_id = int(r[1])

        if (-1 < station_id):
            df_train_cat_one = pd.read_csv(TRAIN_CATEGORICAL,
                                            usecols=cat_feature_one, nrows=NROWS)
            filename_train = 'train_categorical_station_' + str(station_id) + '.csv'
            df_train_cat_one.to_csv(filename_train, index=False)
            print(filename_train)

for cat_feature_one in cat_feature_list:
    if 1 < len(cat_feature_one):
        r = parse("L{}_S{}_F{}", cat_feature_one[1])
        station_id = int(r[1])

        if (-1 < station_id):
            df_test_cat_one  = pd.read_csv(TEST_CATEGORICAL,  
                               usecols=cat_feature_one, nrows=NROWS)
            filename_test  = 'test_categorical_station_'  + str(station_id) + '.csv'
            df_test_cat_one.to_csv(filename_test, index=False)

            print(filename_test)


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (324,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,

train_categorical_station_24.csv


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (984,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,116

train_categorical_station_25.csv


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

test_categorical_station_25.csv


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1552,1554,1556,1558,1560,1562,1564,1566,1568,1570,1572,1574,1576,1578,1580,1582,1584,1586,1589,1591,1593,1595,1597,1599,1601,1603) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


test_categorical_station_26.csv
test_categorical_station_27.csv
test_categorical_station_28.csv
test_categorical_station_29.csv
test_categorical_station_30.csv
test_categorical_station_31.csv
test_categorical_station_32.csv
test_categorical_station_35.csv
test_categorical_station_36.csv
test_categorical_station_38.csv
test_categorical_station_39.csv
test_categorical_station_42.csv
test_categorical_station_43.csv
test_categorical_station_44.csv
test_categorical_station_46.csv


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


test_categorical_station_47.csv
test_categorical_station_49.csv


In [29]:
# feature を抽出してファイルを作る。
for num_feature_one in num_feature_list:
    if 1 < len(num_feature_one):
        r = parse("L{}_S{}_F{}", num_feature_one[1])
        station_id = int(r[1])

        df_train_num_one = pd.read_csv(TRAIN_NUMERIC,
                                        usecols=num_feature_one, nrows=NROWS)
        filename_train = 'train_numeric_station_' + str(station_id) + '.csv'
        df_train_num_one.to_csv(filename_train, index=False)
        print(filename_train)

for num_feature_one in num_feature_list:
    if 1 < len(num_feature_one):
        r = parse("L{}_S{}_F{}", num_feature_one[1])
        station_id = int(r[1])

        df_test_num_one  = pd.read_csv(TEST_NUMERIC,  
                           usecols=num_feature_one, nrows=NROWS)
        filename_test  = 'test_numeric_station_'  + str(station_id) + '.csv'
        df_test_num_one.to_csv(filename_test, index=False)

        print(filename_test)

train_numeric_station_0.csv
train_numeric_station_1.csv
train_numeric_station_2.csv
train_numeric_station_3.csv
train_numeric_station_4.csv
train_numeric_station_5.csv
train_numeric_station_6.csv
train_numeric_station_7.csv
train_numeric_station_8.csv
train_numeric_station_9.csv
train_numeric_station_10.csv
train_numeric_station_11.csv
train_numeric_station_12.csv
train_numeric_station_13.csv
train_numeric_station_14.csv
train_numeric_station_15.csv
train_numeric_station_16.csv
train_numeric_station_17.csv
train_numeric_station_18.csv
train_numeric_station_19.csv
train_numeric_station_20.csv
train_numeric_station_21.csv
train_numeric_station_22.csv
train_numeric_station_23.csv
train_numeric_station_24.csv
train_numeric_station_25.csv
train_numeric_station_26.csv
train_numeric_station_27.csv
train_numeric_station_28.csv
train_numeric_station_29.csv
train_numeric_station_30.csv
train_numeric_station_31.csv
train_numeric_station_32.csv
train_numeric_station_33.csv
train_numeric_station_34

In [ ]:
# 読み出す。

df_train_bin = pd.read_csv(TRAIN_PATH)
df_test_bin  = pd.read_csv(TEST_PATH)
df_pass = pd.concat([df_train_bin, df_test_bin])


通過ステーションをヒートマップにしてみる。